In [77]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [78]:
dim(df_finale)

[1] 1148   78

In [79]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [80]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [81]:
dim(test)
dim(train)

[1] 344 184

[1] 804 184

In [82]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    401     402 

## Model

In [83]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [84]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [85]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [86]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [87]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

803 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.5945272  0.2044221
  1       0.001  0.50  0.7095560  0.4226142
  1       0.001  1.00  0.7281908  0.4580858
  1       0.010  0.25  0.7373920  0.4759291
  1       0.010  0.50  0.7431856  0.4872750
  1       0.010  1.00  0.7427322  0.4862933
  1       0.100  0.25  0.7334580  0.4677272
  1       0.100  0.50  0.7248509  0.4505147
  1       0.100  1.00  0.7191238  0.4387878
  2       0.001  0.25  0.7095789  0.4223839
  2       0.001  0.50  0.7287393  0.4591612
  2       0.001  1.00  0.7350130  0.4713241
  2       0.010  0.25  0.7458048  0.4924177
  2       0.010  0.50  0.7441059  0.4891094
  2       0.010  1.00  0.7413746  0.4835335
  2       0.100  0.25  0.75

In [88]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

803 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.7513102  0.5018756
  0.50  0.7552334  0.5096527
  1.00  0.7615962  0.5225103

Tuning parameter 'sigma' was held constant at a value of 0.01798506
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.01798506 and C = 1.


In [89]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

803 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 803, 803, 803, 803, 803, 803, ... 
Resampling results:

  Accuracy   Kappa    
  0.7126418  0.4249456

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [90]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.   Median      Mean   3rd Qu.      Max. NA's
Poly   0.7162630 0.7440273 0.763754 0.7587756 0.7702703 0.7885305    0
Radial 0.7303754 0.7483444 0.760000 0.7615962 0.7748344 0.8103448    0
Linear 0.6794425 0.6872964 0.707483 0.7126418 0.7322034 0.7711268    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.4391537 0.4860123 0.5266405 0.5182707 0.5392152 0.5775656    0
Radial 0.4530611 0.5012170 0.5184591 0.5225103 0.5496689 0.6138576    0
Linear 0.3563140 0.3705872 0.4177950 0.4249456 0.4648803 0.5421173    0


# Prediction

In [91]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      70      51
   NotFast   60     163
                                          
               Accuracy : 0.6773          
                 95% CI : (0.6251, 0.7265)
    No Information Rate : 0.6221          
    P-Value [Acc > NIR] : 0.01904         
                                          
                  Kappa : 0.3043          
                                          
 Mcnemar's Test P-Value : 0.44766         
                                          
            Sensitivity : 0.5385          
            Specificity : 0.7617          
         Pos Pred Value : 0.5785          
         Neg Pred Value : 0.7309          
             Prevalence : 0.3779          
         Detection Rate : 0.2035          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6501          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5384615            0.7616822            0.5785124 
      Neg Pred Value            Precision               Recall 
           0.7309417            0.5785124            0.5384615 
                  F1           Prevalence       Detection Rate 
           0.5577689            0.3779070            0.2034884 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6500719

In [92]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      83      38
   NotFast   60     163
                                          
               Accuracy : 0.7151          
                 95% CI : (0.6643, 0.7622)
    No Information Rate : 0.5843          
    P-Value [Acc > NIR] : 3.326e-07       
                                          
                  Kappa : 0.4002          
                                          
 Mcnemar's Test P-Value : 0.03389         
                                          
            Sensitivity : 0.5804          
            Specificity : 0.8109          
         Pos Pred Value : 0.6860          
         Neg Pred Value : 0.7309          
             Prevalence : 0.4157          
         Detection Rate : 0.2413          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.6957          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5804196            0.8109453            0.6859504 
      Neg Pred Value            Precision               Recall 
           0.7309417            0.6859504            0.5804196 
                  F1           Prevalence       Detection Rate 
           0.6287879            0.4156977            0.2412791 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.6956824

In [93]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      84      37
   NotFast   55     168
                                          
               Accuracy : 0.7326          
                 95% CI : (0.6825, 0.7786)
    No Information Rate : 0.5959          
    P-Value [Acc > NIR] : 8.127e-08       
                                          
                  Kappa : 0.4329          
                                          
 Mcnemar's Test P-Value : 0.07633         
                                          
            Sensitivity : 0.6043          
            Specificity : 0.8195          
         Pos Pred Value : 0.6942          
         Neg Pred Value : 0.7534          
             Prevalence : 0.4041          
         Detection Rate : 0.2442          
   Detection Prevalence : 0.3517          
      Balanced Accuracy : 0.7119          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.6043165            0.8195122            0.6942149 
      Neg Pred Value            Precision               Recall 
           0.7533632            0.6942149            0.6043165 
                  F1           Prevalence       Detection Rate 
           0.6461538            0.4040698            0.2441860 
Detection Prevalence    Balanced Accuracy 
           0.3517442            0.7119144

In [94]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [95]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 121 controls (TestClasses fast) < 223 cases (TestClasses NotFast).
Area under the curve: 0.7084

In [96]:
pred_svmRadial.prob

fast,NotFast
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
NA,NA
